Notebook elaborado para apresentar uma introdução ao método de Eletrorresistividade.
Criado para a disciplina Geofísica II da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
[emerson.almeida@ufu.br](mailto:emerson.almeida@ufu.br)<br>

Última atualização: 02/10/2022

<p>&nbsp;</p>

---
# Objetivos

- Diferenciar os arranjos Wenner e Schlumberger
- Planejar uma SEV de acordo com a profundidade a ser investigada
- Entender o comportamento da curva de resistividade aparente
- Compreender a diferença entre a resistividade aparente e a resistividade real nos dados de SEV
- Compreender o procedimento da embreagem na sondagem Schlumberger

*Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.*


---
---

### 1. Introdução
Esta atividade utiliza o pacote open source [SimPEG](https://docs.simpeg.xyz/), responsável por executar os cálculos relacionados à física do problema. Se você está executando este notebook *online* não se preocupe, nada será instalado no seu dispositivo local.

**Importante!** Nem todas as células de código desta demostração irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [ ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada.

In [ ]:
import modulos.er1d as er
import numpy as np

### 2. Configuração do modelo geoelétrico

Primeiramente é necessário definir os parâmetros que caracterizam o modelo geoelétrico. Estes parâmetros correspondem à resistividade real e à espessura de cada camada exceto da última, cuja espessura é considerada infinita e não precisa ser inserida no modelo. Lembre-se de que o modelo geoelétrico apresenta apenas os principais contrastes de resistividade, cujas profundidades podem coincidir com as profundidades dos contatos geológicos ou não.

A célula abaixo cria o modelo e apresenta a figura correspondente. Esta é a forma mais comum de se apresentar um modelo geoelétrico, não apenas para sondagens elétricas mas também para sondagens eletromagnéticas.

In [ ]:
# modelo 1D
resistividades = np.r_[50.0, 300.0, 30.0, 1000]
espessuras = np.r_[4.0, 4.0, 30.0]

meu_modelo = er.modelo_geo(resistividades, espessuras)
meu_modelo.mostra_grafico(n_fig='1')

### 3. Aberturas dos eletrodos de corrente (A e B)

A aquisição de uma SEV requer o posicionamento dos eletrodos de injeção de corrente elétrica (A e B) e dos eletrodos de medição de potencial elétrico (M e N). Como os arranjos das SEVs são sempre simétricos é comum definir-se a posição dos eletrodos em relação ao ponto central da sondagem, de forma que define-se a meia distância entre os eletrodos A e B (AB/2) e a meia distância entre os eletrodos M e N (MN/2). O que difere uma SEV em arranjo Schlumberger de uma SEV em arranjo Wenner é o posicionamento dos eletrodos de medida de potencial, enquanto as posições dos eletrodos de injeção de corrente podem iguais para ambos.

A célula abaixo configura as distâncias AB/2 correspondentes às posições dos eletrodos A e B.

**Atividade:** Complete a SEV mantendo as aberturas de AB já inseridas e incluindo mais 15 pontos entra cada uma para amostrar a curva de resistividade aparente. Procure deixar a distribuição dos pontos o mais uniforme possível. Quantos pontos são realmente necessários para representar a curva de forma satisfatória?

In [ ]:
eletrodos_AB = np.array([1.5, 10.0, 100.0, 200.0])

### 4. Configuração dos eletrodos de potencial (M e N) para arranjo Wenner

Neste arranjo as distâncias AM, MN e NB são idênticas, então as posições de M e N podem ser obtidas a partir das posições de A e B ou vice-versa. Aqui iremos calcular as posições de M e N a partir das posições de A e B, o que corresponde às posições de AB/2 e MN/2 em termos de meias-distâncias. Assim, pode-se definir que $(MN)/2 = (1/3) \times (AB)/2$ (verifique graficamente esta relação).

A célula abaixo configura as posições dos eletrodos M e N e apresenta a curva de resistividade aparente correspondente.

**Atividade:** Considere um arranjo Wenner. Complete a SEV calculando as aberturas de MN correspondentes às aberturas de AB que você definiu.

In [ ]:
eletrodos_MN_wenner = np.array([0.50, 3.33, 33.33, 66.67])


sev_w = er.sev(eletrodos_AB, eletrodos_MN_wenner, 'Wenner')
sev_w.fwd(meu_modelo)
er.plota_sondagem(sev_w, meu_modelo, tendencias=False, n_fig='2')

### 5. Configuração dos eletrodos de potencial (M e N) para arranjo Schlumberger

Neste arranjo o afastamento entre os eletrodos M e N deve idealmente obedecer à relação $AB \ge 5 \times MN$, embora em campo seja um pouco difícil manter esta relação para pequenos afastamentos. Nesta demonstração iremos considerar condição limite para a relação entre as disâncias AB e MN, utilizando a relação de $MN/2 = (1/5) \times AB/2$. Em campo costuma-se repetir uma mesma posição de M e N para várias posições de A e B, fazendo desta a principal vantagem logística do arranjo Schlumberger em campo. Em campo utiliza-se como critério para aumento da distância MN o valor do potencial lido, de forma que a distância deve ser aumentada sempre que o potencial lido ficar muito próximo do nível de ruído.

A célula abaixo configura as posições dos eletrodos M e N e apresenta a curva de resistividade aparente correspondente.

**Atividade:** Considere um arranjo Schlumberger. Complete a SEV calculando as aberturas de MN correspondentes às aberturas de AB que você definiu.

In [ ]:
eletrodos_MN_schlumberger = np.array([0.3, 2.0, 20.0, 40.0])


sev_s = er.sev(eletrodos_AB, eletrodos_MN_schlumberger, 'Schlumberger')
sev_s.fwd(meu_modelo)
er.plota_sondagem(sev_s, meu_modelo, tendencias=False, n_fig='3')

### 6. Comparação entre as curvas

A célula abaixo apresenta uma sobreposição das duas curvas simuladas, para que você faça uma comparação entre os dados obtidos com ambos os arranjos. 

**Pergunta:** Como você explica a sutil diferença entre elas se o modelo geoelétrico é o mesmo? Isso deveria ser esperado?

In [ ]:
er.compara_sevs(sev_s, sev_w, n_fig='4')

### 7. Componentes gráficos adicionais

A célula abaixo plota a mesma curva de dados modelada com o arranjo Schlumberger no item 5, porém adicionando curvas adicionais junto à curva de dados de resistividade aparente. Estas curvas adicionais correspondem à curva de resistividade aparente considerando diferentes números de camadas ($n$).

**Atividade:** Faça uma análise simulando o modelo 1D do Item 02 com 1, 2, 3 e 4 camadas. Explique o comportamento destas linhas em relação à curva de dados da SEV e em relação ao modelo geoelétrico.

In [ ]:
sev_s = er.sev(eletrodos_AB, eletrodos_MN_schlumberger, 'Schlumberger')
sev_s.fwd(meu_modelo)
er.plota_sondagem(sev_s, meu_modelo, tendencias=True, n_fig='3')